I created a new kind of mpohamiltonian for quantum chemistry hamiltonians, because you can gain some speed there by pre-summing the environments. I also invested a bunch of effort making the effective hamiltonian fast. In this notebook I try to benchmark the MPSKit effective hamiltonians against my (?optimized) implementation.

I don't make use of the pre-summation in this benchmark, both codes should do the exact same thing.

In [1]:
using Revise, MPSKit, MPSKitModels, TensorKit
using LinearAlgebra, Base.Threads, BenchmarkTools
using MPSKitExperimental:FusedMPOHamiltonian

In [2]:
BLAS.set_num_threads(1)

In [3]:
nthreads()

4

## heisenberg spin 1

In [4]:
fmps_len = 10;
middle_site = Int(round(fmps_len/2))

ts = FiniteMPS(rand,ComplexF64,fmps_len,Rep[SU₂](1=>1),Rep[SU₂](i => 20 for i in 0:10));

th_orig = heisenberg_XXX(SU2Irrep);
th_fused = convert(FusedMPOHamiltonian,repeat(th_orig,length(ts)));

env_orig = environments(ts,th_orig);
env_fused = environments(ts,th_fused);

ac = ts.AC[middle_site]
ac_eff_orig = MPSKit.∂∂AC(middle_site,ts,th_orig,env_orig);
ac_eff_fused = MPSKit.∂∂AC(middle_site,ts,th_fused,env_fused);


ac2 = ts.AC[middle_site]*MPSKit._transpose_tail(ts.AR[middle_site+1])
ac2_eff_orig = MPSKit.∂∂AC2(middle_site,ts,th_orig,env_orig);
ac2_eff_fused = MPSKit.∂∂AC2(middle_site,ts,th_fused,env_fused);

In [5]:
@benchmark ac_eff_orig(ac)

BenchmarkTools.Trial: 3028 samples with 1 evaluation.
 Range (min … max):  859.053 μs … 13.965 ms  ┊ GC (min … max):  0.00% … 80.16%
 Time  (median):       1.368 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.643 ms ±  1.638 ms  ┊ GC (mean ± σ):  13.85% ± 12.20%

  ▁▅█▅▁                                                         
  █████▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂ ▃
  859 μs          Histogram: frequency by time           12 ms <

 Memory estimate: 1.43 MiB, allocs estimate: 14410.

In [6]:
@benchmark ac_eff_fused(ac)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   34.610 μs …  16.455 ms  ┊ GC (min … max): 0.00% … 97.11%
 Time  (median):      88.554 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   105.244 μs ± 402.377 μs  ┊ GC (mean ± σ):  9.71% ±  2.54%

           ▄▆█▇▆▆▇▅▅▄▃▃▁                                         
  ▃▆▄▂▃▄▅▅▆██████████████▆▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  34.6 μs          Histogram: frequency by time          245 μs <

 Memory estimate: 123.69 KiB, allocs estimate: 971.

In [7]:
@benchmark ac2_eff_orig(ac2)

BenchmarkTools.Trial: 2227 samples with 1 evaluation.
 Range (min … max):  1.316 ms … 24.615 ms  ┊ GC (min … max):  0.00% … 87.28%
 Time  (median):     1.787 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.237 ms ±  3.018 ms  ┊ GC (mean ± σ):  19.29% ± 12.93%

  ▇█▅▂                                                        
  ████▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▃▅▃▇▇ █
  1.32 ms      Histogram: log(frequency) by time     22.3 ms <

 Memory estimate: 3.55 MiB, allocs estimate: 35055.

In [8]:
@benchmark ac2_eff_fused(ac2)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  39.529 μs …  17.418 ms  ┊ GC (min … max): 0.00% … 95.38%
 Time  (median):     78.534 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   93.214 μs ± 297.227 μs  ┊ GC (mean ± σ):  5.30% ±  1.66%

             ▂▄█▂                                               
  ▁▂▂▂▂▃▃▂▂▂▄████▆▄▄▄▅▅▅▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  39.5 μs         Histogram: frequency by time          191 μs <

 Memory estimate: 75.88 KiB, allocs estimate: 1004.

## some weird hamiltonian

In [9]:
using MAT
include("haagerup/H3_anyon.jl")

In [10]:
fmps_len = 10;
middle_site = Int(round(fmps_len/2))

physical = Vect[H3](H3(4)=>1);
virtual = Vect[H3](sector => 20 for sector in TensorKit.SectorValues{H3}());
ts = FiniteMPS(rand,ComplexF64,fmps_len,physical,virtual);

mpotensor = TensorMap(ones,ComplexF64,physical*physical,physical*physical);
blocks(mpotensor)[H3(1)]*=0;
blocks(mpotensor)[H3(5)]*=0;
blocks(mpotensor)[H3(6)]*=0;
th_orig = MPOHamiltonian(-mpotensor);
th_fused = convert(FusedMPOHamiltonian,repeat(th_orig,length(ts)));

env_orig = environments(ts,th_orig);
env_fused = environments(ts,th_fused);

ac = ts.AC[middle_site]
ac_eff_orig = MPSKit.∂∂AC(middle_site,ts,th_orig,env_orig);
ac_eff_fused = MPSKit.∂∂AC(middle_site,ts,th_fused,env_fused);


ac2 = ts.AC[middle_site]*MPSKit._transpose_tail(ts.AR[middle_site+1])
ac2_eff_orig = MPSKit.∂∂AC2(middle_site,ts,th_orig,env_orig);
ac2_eff_fused = MPSKit.∂∂AC2(middle_site,ts,th_fused,env_fused);

In [11]:
@benchmark ac_eff_orig(ac)

BenchmarkTools.Trial: 747 samples with 1 evaluation.
 Range (min … max):  3.480 ms … 28.923 ms  ┊ GC (min … max):  0.00% … 75.71%
 Time  (median):     4.967 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.684 ms ±  4.785 ms  ┊ GC (mean ± σ):  24.63% ± 23.09%

   ▁▄██▄▁                                                     
  ▄██████▅▄▄▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▃▃▃▂▃▂▂▂▂▁▁▂ ▃
  3.48 ms        Histogram: frequency by time        22.3 ms <

 Memory estimate: 6.93 MiB, allocs estimate: 52565.

In [12]:
@benchmark ac_eff_fused(ac)

BenchmarkTools.Trial: 7108 samples with 1 evaluation.
 Range (min … max):  463.256 μs …  16.480 ms  ┊ GC (min … max): 0.00% … 92.75%
 Time  (median):     618.374 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   696.599 μs ± 685.468 μs  ┊ GC (mean ± σ):  4.42% ±  4.40%

    ▁▄▆██▆▄▂▂               ▁ ▁▁                                 
  ▅███████████▇▅▆▅▄▃▃▃▃▄▄▄▆▇███████▆▆▄▄▃▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁ ▄
  463 μs           Histogram: frequency by time         1.12 ms <

 Memory estimate: 405.84 KiB, allocs estimate: 1702.

In [13]:
@benchmark ac2_eff_orig(ac2)

BenchmarkTools.Trial: 266 samples with 1 evaluation.
 Range (min … max):  12.984 ms … 36.734 ms  ┊ GC (min … max):  0.00% … 57.73%
 Time  (median):     14.589 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.823 ms ±  8.341 ms  ┊ GC (mean ± σ):  22.97% ± 23.63%

   ▃██▂▃                                                       
  ▅█████▆▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▃▄▃▅▄▄▄▂▃▃ ▃
  13 ms           Histogram: frequency by time        36.5 ms <

 Memory estimate: 33.43 MiB, allocs estimate: 324725.

In [14]:
@benchmark ac2_eff_fused(ac2)

BenchmarkTools.Trial: 4869 samples with 1 evaluation.
 Range (min … max):  804.375 μs … 21.560 ms  ┊ GC (min … max): 0.00% … 90.57%
 Time  (median):     897.784 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.017 ms ±  1.048 ms  ┊ GC (mean ± σ):  6.60% ±  6.10%

    ▅▇█▅▃▁                                                      
  ▅███████▇▇▅▅▅▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂ ▃
  804 μs          Histogram: frequency by time         1.74 ms <

 Memory estimate: 377.53 KiB, allocs estimate: 4019.

## manueel

manually written out @tensor, compared to a version that used DelayedFact & friends. The whole recycling of intermediaries becomes mostly important when multithreading, so not very applicable here.

In [15]:
using MPSKitExperimental:DelayedFact,TransposeFact,free!

In [16]:
virtual = Rep[SU₂](i => 20 for i in 0:10);
physical = Rep[SU₂](1=>1)

ac = TensorMap(rand,ComplexF64,virtual*physical,virtual)

fun = let        
    e = transpose(TensorMap(rand,ComplexF64,oneunit(physical)*physical,physical*oneunit(physical)),(1,),(3,4,2))
    l = transpose(TensorMap(rand,ComplexF64,virtual*oneunit(physical)',virtual),(3,1),(2,))
    le = transpose(l*e,(2,5,4),(1,3))

    r = TensorMap(rand,ComplexF64,virtual*oneunit(physical),virtual)


    function tobench(x)
        @plansor y[-1 -2;-3] := le[-1 -2 5;2 3]*x[2 3;4]*r[4 5;-3]
    end
end

@benchmark fun(ac)

BenchmarkTools.Trial: 5192 samples with 1 evaluation.
 Range (min … max):  548.926 μs … 14.786 ms  ┊ GC (min … max):  0.00% … 90.50%
 Time  (median):     704.005 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   955.215 μs ±  1.664 ms  ┊ GC (mean ± σ):  23.67% ± 12.61%

  █▇▄                                                          ▁
  ████▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▆▇▇ █
  549 μs        Histogram: log(frequency) by time      12.5 ms <

 Memory estimate: 974.44 KiB, allocs estimate: 4094.

In [17]:
virtual = Rep[SU₂](i => 20 for i in 0:10);
physical = Rep[SU₂](1=>1)

ac = TensorMap(rand,ComplexF64,virtual*physical,virtual)

fun = let        
    e = transpose(TensorMap(rand,ComplexF64,oneunit(physical)*physical,physical*oneunit(physical)),(1,),(3,4,2))
    l = transpose(TensorMap(rand,ComplexF64,virtual*oneunit(physical)',virtual),(3,1),(2,))
    le = transpose(l*e,(2,5,4),(1,3))

    r = TensorMap(rand,ComplexF64,virtual*oneunit(physical),virtual)


    function tobench(x)

        lex = le*x
        lex_trans = transpose(lex,(1,2),(4,3))
        lex_trans*r
    end
end

@benchmark fun(ac)

BenchmarkTools.Trial: 6814 samples with 1 evaluation.
 Range (min … max):  457.697 μs … 13.747 ms  ┊ GC (min … max):  0.00% … 91.21%
 Time  (median):     523.471 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   725.849 μs ±  1.418 ms  ┊ GC (mean ± σ):  23.83% ± 11.48%

  █▄                                                           ▁
  ███▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▆▇ █
  458 μs        Histogram: log(frequency) by time      11.6 ms <

 Memory estimate: 771.36 KiB, allocs estimate: 2589.

In [18]:
virtual = Rep[SU₂](i => 20 for i in 0:10);
physical = Rep[SU₂](1=>1)

ac = TensorMap(rand,ComplexF64,virtual*physical,virtual)

fun = let        
    e = transpose(TensorMap(rand,ComplexF64,oneunit(physical)*physical,physical*oneunit(physical)),(1,),(3,4,2))
    l = transpose(TensorMap(rand,ComplexF64,virtual*oneunit(physical)',virtual),(3,1),(2,))
    le = transpose(l*e,(2,5,4),(1,3))

    r = TensorMap(rand,ComplexF64,virtual*oneunit(physical),virtual)

    multfactory_lex = DelayedFact(codomain(le)←virtual,storagetype(r))
    transposefactor_lex = TransposeFact(codomain(le)←virtual,storagetype(r),(1,2),(4,3))
    multfact_out = DelayedFact(virtual*physical←virtual,storagetype(r))

    function tobench(x)
        t_1 = multfactory_lex()
        mul!(t_1,le,x)

        t_2 = transposefactor_lex(t_1)
        free!(multfactory_lex,t_1);

        y = multfact_out()

        mul!(y,t_2,r)

        free!(transposefactor_lex,t_2)
        y
    end
end

@benchmark fun(ac)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  233.359 μs …  15.651 ms  ┊ GC (min … max):  0.00% … 94.49%
 Time  (median):     246.359 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   309.244 μs ± 731.047 μs  ┊ GC (mean ± σ):  15.04% ±  6.25%

  ▄▇█▇▆▅▄▄▄▄▄▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁                                   ▂
  ████████████████████████████████▇██▇████▇▇█▇▆▇▇▆▆▇▅▆▅▆▆▅▆▆▃▅▄ █
  233 μs        Histogram: log(frequency) by time        419 μs <

 Memory estimate: 198.03 KiB, allocs estimate: 227.

## Team Topo

In [19]:
# original

using TensorKit, TensorOperations
using BenchmarkTools

tensorexpr(name::Symbol, inds) = Expr(:ref, name, inds...)
@generated function apply_transfer2(A, x::AbstractTensorMap{S,N,0}) where {S,N}
    N > 0 || error("undefined behaviour for length 0")

    out_part = tensorexpr(:y, -1:-1:-N)

    in_part = Expr(:call, :*, tensorexpr(:x, 1:2:2N), tensorexpr(:A, (2N, -1, 1, 2)))
    for i = 2:N
        push!(in_part.args, tensorexpr(:A, (2i - 2, -i,  2i - 1, 2i)))
    end

    return :(@tensor $out_part := $in_part)
end


println(Threads.nthreads())
for n = 3:5
    println("tests for n = $n")
    let N = n, sp = ℂ^2
        A = permute(TensorMap(rand, ComplexF64, sp^2 ← sp^2),(4,1),(3,2))
        X = Tensor(rand, ComplexF64, (sp)^N)
        @btime apply_transfer2($A, $X)
        nothing
    end

    let N = n, sp = Z2Space(0 => 1, 1 => 1)
        A = permute(TensorMap(rand, ComplexF64, sp^2 ← sp^2),(4,1),(3,2))
        X = Tensor(rand, ComplexF64, (sp)^N)
        @btime apply_transfer2($A, $X)
        nothing
    end
end

4
tests for n = 3


  15.569 μs (88 allocations: 10.17 KiB)


  114.559 μs (2504 allocations: 169.06 KiB)
tests for n = 4


  37.380 μs (126 allocations: 18.50 KiB)


  198.259 μs (4891 allocations: 367.80 KiB)
tests for n = 5


  64.930 μs (172 allocations: 33.98 KiB)


  384.678 μs (10019 allocations: 859.91 KiB)


In [18]:
# now with transposefactory
using ProfileView
using PProf
function apply_transfer_fast(A,x::AbstractTensorMap{S,N,0}) where {S,N}
    N > 0 || error("undefined behaviour for length 0")

    trans_x_first = TransposeFact(codomain(x)←domain(x),storagetype(x),(1,),tuple(N:-1:2...))
    trans_A_first = TransposeFact(codomain(A)←domain(A),storagetype(A),(1,2,4),(3,))
    
    multcache_1 = DelayedFact(trans_A_first.delayed.cod←trans_x_first.delayed.dom,storagetype(x))

    trans_AX_first = TransposeFact(multcache_1,(3,N+2),(2,1,4:N+1...))

    trans_A_middle = TransposeFact(codomain(A)←domain(A),storagetype(A),(2,4),(1,3))
    multcache_middle = DelayedFact(trans_A_middle.delayed.cod←prod(space(x,1)' for i in 1:N),storagetype(A))
    trans_AX_middle = TransposeFact(multcache_middle.cod←multcache_middle.dom,storagetype(A),(2,N+2),(1,3:N+1...))

    function ff(A,x)
        for top in 1:1000
            t_A1 = trans_A_first(A)
            t_x1 = trans_x_first(x)
            
            t_Ax1 = multcache_1()
            mul!(t_Ax1,t_A1,t_x1)
            free!(trans_A_first,t_A1)
            free!(trans_x_first,t_x1)

            t_Ax2 = trans_AX_first(t_Ax1)
            free!(multcache_1,t_Ax1)

            for i in 2:N-1
                tA = trans_A_middle(A)
                t_m = multcache_middle()
                mul!(t_m,tA,t_Ax2)
                
                free!(trans_A_middle,tA)
                if i == 2
                    free!(trans_AX_first,t_Ax2)
                else
                    free!(trans_AX_middle,t_Ax2)
                end

                t_Ax2 = trans_AX_middle(t_m)
                free!(multcache_middle,t_m)
            end
        end
    end

end

println(Threads.nthreads())
n = 4
using Profile
Profile.clear()
let N = n, sp = Z2Space(0 => 1, 1 => 1)
    A = TensorMap(rand, ComplexF64, sp*sp ← sp*sp)
    X = Tensor(rand, ComplexF64, (sp)^N)
    fastfun = apply_transfer_fast(A,X)
    #@btime $fastfun($A, $X)
    fastfun(A,X)
    #VSCodeServer.@profview fastfun(A,X)
    ProfileView.@profview fastfun(A,X)
end

4


FATAL ERROR: Gtk state corrupted by error thrown in a callback:



FATAL ERROR: Gtk state corrupted by error thrown in a callback:
FATAL ERROR: Gtk state corrupted by error thrown in a callback:


ERROR: AssertionError: g_stack === nothing && !prev
Stacktrace:
 [1] g_sigatom(f::Any)
   @ Gtk.GLib ~/.julia/packages/Gtk/oo3cW/src/GLib/signals.jl:174
 [2] gtk_main()
   @ Gtk ~/.julia/packages/Gtk/oo3cW/src/events.jl:1
ERROR: AssertionError: g_stack === nothing && !prevERROR: 
Stacktrace:


  [1] AssertionError: g_sigatomg_stack === nothing && !prev(
Stacktrace:f
:: Any[1]) 
g_sigatom    @( fGtk.GLib:: Any~/.julia/packages/Gtk/oo3cW/src/GLib/)signals.jl:174
   @
  Gtk.GLib [2]  ~/.julia/packages/Gtk/oo3cW/src/GLib/macro expansionsignals.jl:174
    @
  ~/.julia/packages/Gtk/oo3cW/src/GLib/[2]signals.jl:200  [inlined]gtk_main
( ) [3]
    @show 
Gtk    @  ~/.julia/packages/Gtk/oo3cW/src/~/.julia/packages/Gtk/oo3cW/src/events.jl:1base.jl:39 [inlined]

  [4] Gtk.GtkWindowLeaf

(title::String, w::Int64, h::Int64, resizable::Bool, toplevel::Bool)
    @ Gtk ~/.julia/packages/Gtk/oo3cW/src/windows.jl:14
  [5] GtkWindowLeaf
    @ ~/.julia/packages/Gtk/oo3cW/src/windows.jl:2 [inlined]
  [6] #GtkWindow#196
    @ ~/.julia/packages/Gtk/oo3cW/src/GLib/gtype.jl:226 [inlined]
  [7] GtkWindow
    @ ~/.julia/packages/Gtk/oo3cW/src/GLib/gtype.jl:225 [inlined]
  [8] viewgui(fcolor::FlameGraphs.FlameColors, gdict::Dict{Symbol, Dict{Symbol, LeftChildRightSiblingTrees.Node{FlameGraphs.NodeData}}}; data::Vector{UInt64}, lidict::Dict{UInt64, Vector{Base.StackTraces.StackFrame}}, windowname::String, graphtype::Symbol, kwargs::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
    @ ProfileView ~/.julia/packages/ProfileView/IzPaD/src/ProfileView.jl:310
  [9] viewgui
    @ ~/.julia/packages/ProfileView/IzPaD/src/ProfileView.jl:230 [inlined]
 [10] #view#13
    @ ~/.julia/packages/ProfileView/IzPaD/src/ProfileView.jl:222 [inlined]
 [11] view(fcolor::FlameGraphs.FlameColor

error in running finalizer: AssertionError(msg="xor(prev, current_task() !== g_stack)")
g_siginterruptible at /home/maarten/.julia/packages/Gtk/oo3cW/src/GLib/signals.jl:209
unknown function (ip: 0x7fce8fb69f36)
GClosureMarshal at /home/maarten/.julia/packages/Gtk/oo3cW/src/GLib/signals.jl:58
unknown function (ip: 0x7fce8fb55f8a)
jlcapi_GClosureMarshal_1208 at /home/maarten/.julia/compiled/v1.9/Gtk/Vjnq0_ClZO7.so (unknown line)
g_closure_invoke at /home/maarten/.julia/artifacts/b6ebc4def1211ec4043d7c055f450d59f56747cc/lib/libgobject-2.0.so.0 (unknown line)
signal_emit_unlocked_R at /home/maarten/.julia/artifacts/b6ebc4def1211ec4043d7c055f450d59f56747cc/lib/libgobject-2.0.so.0 (unknown line)
g_signal_emit_valist at /home/maarten/.julia/artifacts/b6ebc4def1211ec4043d7c055f450d59f56747cc/lib/libgobject-2.0.so.0 (unknown line)
g_signal_emit at /home/maarten/.julia/artifacts/b6ebc4def1211ec4043d7c055f450d59f56747cc/lib/libgobject-2.0.so.0 (unknown line)
gtk_widget_dispose at /workspace/srcd

AssertionError: AssertionError: g_stack === nothing && !prev